In [1]:
# === 初始化环境 ===
import sys
import os
sys.path.append(os.path.abspath('../'))  # 添加项目根目录到系统路径

from datetime import datetime, timedelta
from scraper import create_session, scrape_single_date, format_url_date
from storage import HlDataSaver, HlDataLoader
import logging
import json

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')

# === 初始化组件 ===
session = create_session()  # 创建网络会话
data_file = '../huangli_data.json'  # 数据文件路径

In [7]:
# === 批量抓取示例 ===
def demo_batch_scrape():
    print("\n=== 批量抓取演示 ===")
    start_date = datetime(2025, 3, 3)
    end_date = datetime(2025, 3, 3)
    
    # 生成日期列表
    date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    target_dates = [format_url_date(d) for d in date_list]
    
    # 初始化存储
    saver = HlDataSaver(data_file)
    
    # 执行批量抓取
    success_count = 0
    for date_str in target_dates:
        try:
            data = scrape_single_date(date_str, session)
            if data:
                saver.save_incrementally([data])
                success_count += 1
                print(f"√ {date_str} 抓取成功")
            else:
                print(f"× {date_str} 抓取失败")
        except Exception as e:
            logging.error(f"{date_str} 抓取异常: {str(e)}")
    
    print(f"\n完成！成功抓取 {success_count}/{len(target_dates)} 天数据")

demo_batch_scrape()


=== 批量抓取演示 ===
√ 2025-3-3 抓取成功

完成！成功抓取 1/1 天数据


In [9]:
# === 数据查询示例 ===
def demo_data_query():
    print("\n=== 数据查询演示 ===")
    loader = HlDataLoader(data_file)
    
    # 单日查询
    test_date = "2025-03-03"
    data = loader.get_by_date(test_date)
    if data:
        print(f"{test_date} 查询结果：")
        print(json.dumps({
            "日期": data["datekey"],
            "星座": data["constellation"],
            "幸运数字": data["lucky_num"],
            "吉时": data["lucky_time"]
        }, indent=2, ensure_ascii=False))
    else:
        print("未找到该日期数据")
    
    # 范围查询
    print("\n三月第一周数据：")
    results = loader.filter_data(
        start_date=datetime(2025,3,1),
        end_date=datetime(2025,3,7)
    )
    print(f"共找到 {len(results)} 条记录")
    for item in results:
        print(f"{item['datekey']}: {item['zs']}")

demo_data_query()


=== 数据查询演示 ===
2025-03-03 查询结果：
{
  "日期": "2025/03/03 00:00:00",
  "星座": "双鱼座",
  "幸运数字": "5、0",
  "吉时": "庚寅时 03:00-04:59, 辛卯时 05:00-06:59, 癸巳时 09:00-10:59, 丙申时 15:00-16:59, 戊戌时 19:00-20:59, 己亥时 21:00-22:59"
}

三月第一周数据：
共找到 7 条记录
2025/03/01 00:00:00: 天德
2025/03/02 00:00:00: 白虎
2025/03/03 00:00:00: 玉堂
2025/03/04 00:00:00: 天牢
2025/03/05 00:00:00: 玉堂
2025/03/06 00:00:00: 天牢
2025/03/07 00:00:00: 玄武


In [9]:
# === 数据统计示例 ===
def demo_statistics():
    print("\n=== 数据统计演示 ===")
    loader = HlDataLoader(data_file)
    
    stats = loader.get_statistics()
    print(f"总数据量: {stats['total_days']} 天")
    print("\n最常见的5个吉神宜趋：")
    for idx, (item, count) in enumerate(stats['most_common_jsyq'][:5], 1):
        print(f"{idx}. {item} ({count}次)")
    
    # 数据校验
    print("\n数据完整性检查：")
    errors = loader.validate_data()
    if not errors:
        print("✅ 所有数据完整有效")
    else:
        print(f"发现 {len(errors)} 个问题：")
        for error in errors[:3]:  # 只显示前3个错误
            print(f"⚠ {error}")

demo_statistics()


=== 数据统计演示 ===
总数据量: 1 天

最常见的5个吉神宜趋：
1. 相日 宝光 (1次)

数据完整性检查：
✅ 所有数据完整有效
